# **Hybrid Classical + Quantum Cryptographic Communication System**

Working hybrid crypto simulation that:

    1) Models basic QKD noise and error.

    2) Evaluates quantum channel health via entropy.

    3) Performs fallback route logic based on detected anomalies.

    4) Uses real symmetric encryption (AES) like a post-quantum hybrid might.

In [1]:
# Install required libraries
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.8 MB/s eta 0:00:00


In [3]:

import random
import hashlib
import base64
import time
import math
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# --- 1. Simulated QKD Key Exchange ---
def simulate_qkd_key(length=16, error_prob=0.05):
    key = []
    for _ in range(length):
        bit = random.choice([0, 1])
        # Simulate quantum error
        if random.random() < error_prob:
            bit ^= 1  # Bit flip
        key.append(str(bit))
    return ''.join(key)

# --- 2. Convert QKD Key to AES Key ---
def qkd_to_aes_key(qkd_key):
    # Hash QKD key to 128-bit AES key
    return hashlib.sha256(qkd_key.encode()).digest()[:16]

# --- 3. PQC-style AES Encryption ---
def encrypt_message(message, key):
    cipher = AES.new(key, AES.MODE_EAX)
    ciphertext, tag = cipher.encrypt_and_digest(message.encode())
    return {
        'ciphertext': base64.b64encode(ciphertext).decode(),
        'nonce': base64.b64encode(cipher.nonce).decode(),
        'tag': base64.b64encode(tag).decode()
    }

def decrypt_message(encrypted, key):
    cipher = AES.new(key, AES.MODE_EAX, nonce=base64.b64decode(encrypted['nonce']))
    plaintext = cipher.decrypt_and_verify(
        base64.b64decode(encrypted['ciphertext']),
        base64.b64decode(encrypted['tag'])
    )
    return plaintext.decode()

# --- 4. Corrected Entropy Calculation ---
def calculate_entropy(bits):
    p0 = bits.count('0') / len(bits)
    p1 = bits.count('1') / len(bits)
    entropy = 0
    if p0 > 0:
        entropy -= p0 * math.log2(p0)
    if p1 > 0:
        entropy -= p1 * math.log2(p1)
    return round(entropy, 3)

def detect_anomaly(qkd_key):
    entropy = calculate_entropy(qkd_key)
    print(f"🧪 QKD Key Entropy: {entropy}")
    if entropy < 0.9 or entropy > 1.1:
        print("⚠️  Anomaly Detected: Channel integrity compromised.")
        return True
    return False

# --- 5. Simulated Routing Logic ---
def simulate_routing(qkd_valid=True):
    if qkd_valid:
        return "Quantum Route → Secure Transmission"
    else:
        return "Classical Route → PQC-Only Fallback"

# --- 6. Run Demo ---
def run_demo(user_message, error_prob=0.1):
    print("🔐 Starting QKD Session...")
    qkd_key = simulate_qkd_key(error_prob=error_prob)
    aes_key = qkd_to_aes_key(qkd_key)

    print("🔄 Checking Key Integrity...")
    anomaly = detect_anomaly(qkd_key)

    route = simulate_routing(not anomaly)
    print(f"📡 Selected Route: {route}")

    print("🛡️ Encrypting Message...")
    encrypted = encrypt_message(user_message, aes_key)
    print(f"📨 Ciphertext (base64): {encrypted['ciphertext'][:64]}...")

    print("🔓 Decrypting Message...")
    try:
        decrypted = decrypt_message(encrypted, aes_key)
        print(f"✅ Decrypted: {decrypted}")
    except Exception as e:
        print("❌ Decryption Failed. Reason:", str(e))

# === Run the Hybrid System Demo ===
run_demo("Quantum secure message: Mission Sync Code Zeta", error_prob=0.1)


🔐 Starting QKD Session...
🔄 Checking Key Integrity...
🧪 QKD Key Entropy: 0.989
📡 Selected Route: Quantum Route → Secure Transmission
🛡️ Encrypting Message...
📨 Ciphertext (base64): HewIQv2UqsBLjdTmWGpthPkrQdkWW/W2Y5OH2kdKeuhITuszjr+YK//0+bmC7g==...
🔓 Decrypting Message...
✅ Decrypted: Quantum secure message: Mission Sync Code Zeta


**Concept and execution by Bhadale IT, code generated by ChatGPT**